In [69]:
import pandas as pd
import numpy as np
import csv
import os

setting up the data 

In [70]:
def pair_match_point_files(file_names):
    file_set = set(file_names)
    pairs = []
    for name in file_names:
        if "matches" in name:
            points_name = name.replace("matches", "points")
            if points_name in file_set:
                pairs.append([name, points_name])
    return pairs

In [71]:
files = pair_match_point_files(os.listdir('data/raw data'))
exp = files[0][1]

In [72]:
def _safe_div(a, b):
    a = a.astype(float)
    b = b.astype(float)
    out = np.divide(a, b, out=np.zeros_like(a, dtype=float), where=(b!=0))
    return out

def _is_game_point_for(pts_for, pts_against, is_tiebreak):
    # Standard (non-TB) tennis game: win at >=4 and lead >=2.
    if is_tiebreak:
        return False
    # If you currently have 3 (i.e., 40) and opp <=2, winning this point gives 4 vs <=3 (lead >=2)
    cond1 = (pts_for == 3) & (pts_against <= 2)
    # If deuce or beyond: you have advantage (lead exactly 1); winning gives lead 2
    cond2 = (pts_for >= 3) & (pts_against >= 3) & (pts_for == pts_against + 1)
    return cond1 | cond2

def _is_game_point_against(pts_for, pts_against, is_tiebreak):
    # symmetric
    return _is_game_point_for(pts_against, pts_for, is_tiebreak)




In [73]:
def build_match_state_panel(input_data, best_of_default=5):
    """
    Returns a DataFrame with one row per point per perspective (P1/P2),
    containing the scoreboard state BEFORE the point and pressure flags.
    """
    df = input_data
    # Ensure required columns exist
    needed = ['match_id','SetNo','GameNo','PointNumber','PointServer']
    for c in needed:
        if c not in df.columns:
            raise ValueError(f"Missing required column: {c}")
    # Try to get point winner (+ robust fallback)
    if 'PointWinner' in df.columns:
        # 1 or 2 (winner of the point)
        pw = df['PointWinner']
        p1_won_point = (pw == 1).astype(int)
        p2_won_point = (pw == 2).astype(int)
    else:
        # Fallback if PointWinner absent: infer from cumulative P1PointsWon/P2PointsWon
        if {'P1PointsWon','P2PointsWon'}.issubset(df.columns):
            g = df.sort_values(['match_id','SetNo','GameNo','PointNumber']).copy()
            g['p1_cum_prev'] = g.groupby('match_id')['P1PointsWon'].shift(1).fillna(0)
            g['p2_cum_prev'] = g.groupby('match_id')['P2PointsWon'].shift(1).fillna(0)
            p1_won_point = (g['P1PointsWon'] > g['p1_cum_prev']).astype(int)
            p2_won_point = (g['P2PointsWon'] > g['p2_cum_prev']).astype(int)
            df = g  # keep the sorted/augmented frame
        else:
            raise ValueError("Need either PointWinner or (P1PointsWon,P2PointsWon) to derive point winners.")

    # Sort and add a per-match point index
    df = df.sort_values(['match_id','SetNo','GameNo','PointNumber']).copy()
    df['point_idx'] = df.groupby('match_id').cumcount() + 1

    # ---------- GAME-LEVEL POINT COUNTS (before current point) ----------
    # Mark game boundaries
    df['game_key'] = df.groupby('match_id').apply(
        lambda g: (g['SetNo'].astype(str) + '-' + g['GameNo'].astype(str)).ne(
            (g['SetNo'].astype(str) + '-' + g['GameNo'].astype(str)).shift(1)
        ).cumsum()
    ).reset_index(level=0, drop=True)

    # Points won per game (cumulative), then shift to be "state BEFORE point"
    df['p1_pts_in_game'] = df.groupby(['match_id','game_key'])[p1_won_point.name if p1_won_point.name else 'PointNumber'].transform(lambda s: s*0)
    df['p2_pts_in_game'] = df['p1_pts_in_game']  # placeholders to allocate cols

    p1_cum_in_game = df.groupby(['match_id','game_key'])[p1_won_point.index.names[-1] if hasattr(p1_won_point, 'index') else p1_won_point].transform(lambda _: 0)  # just to keep structure

    # Actually compute with cumsums then shift:
    df['p1_pts_in_game'] = df.groupby(['match_id','game_key'])[p1_won_point.name if p1_won_point.name else 'point_idx'].apply(
        lambda s: (s.index*0)  # dummy; we recompute below
    ).reset_index(level=[0,1], drop=True)

    # Easier: recompute directly
    df['p1_pts_in_game'] = p1_won_point.groupby([df['match_id'], df['game_key']]).cumsum().shift(1).fillna(0).astype(int)
    df['p2_pts_in_game'] = p2_won_point.groupby([df['match_id'], df['game_key']]).cumsum().shift(1).fillna(0).astype(int)

    # ---------- MATCH-LEVEL CUMULATIVES (TTL etc., before current point) ----------
    df['ttl_p1'] = p1_won_point.groupby(df['match_id']).cumsum().shift(1).fillna(0).astype(int)
    df['ttl_p2'] = p2_won_point.groupby(df['match_id']).cumsum().shift(1).fillna(0).astype(int)

    # Optional serve/return counters if present → rates (before current point)
    have_srv_cols = {'P1FirstSrvIn','P1FirstSrvWon','P1SecondSrvIn','P1SecondSrvWon','P1DoubleFault',
                     'P2FirstSrvIn','P2FirstSrvWon','P2SecondSrvIn','P2SecondSrvWon','P2DoubleFault'}.issubset(df.columns)
    if have_srv_cols:
        for side in (1,2):
            df[f'p{side}_fs_in_cum']  = df[f'P{side}FirstSrvIn'].groupby(df['match_id']).cumsum().shift(1).fillna(0)
            df[f'p{side}_fs_won_cum'] = df[f'P{side}FirstSrvWon'].groupby(df['match_id']).cumsum().shift(1).fillna(0)
            df[f'p{side}_ss_in_cum']  = df[f'P{side}SecondSrvIn'].groupby(df['match_id']).cumsum().shift(1).fillna(0)
            df[f'p{side}_ss_won_cum'] = df[f'P{side}SecondSrvWon'].groupby(df['match_id']).cumsum().shift(1).fillna(0)
            df[f'p{side}_df_cum']     = df[f'P{side}DoubleFault'].groupby(df['match_id']).cumsum().shift(1).fillna(0)
        df['p1_sv_pts'] = df['p1_fs_in_cum'] + df['p1_ss_in_cum'] + df['p1_df_cum']
        df['p2_sv_pts'] = df['p2_fs_in_cum'] + df['p2_ss_in_cum'] + df['p2_df_cum']
        df['p1_fsp'] = _safe_div(df['p1_fs_in_cum'], df['p1_sv_pts'])
        df['p2_fsp'] = _safe_div(df['p2_fs_in_cum'], df['p2_sv_pts'])
        df['p1_w1sp'] = _safe_div(df['p1_fs_won_cum'], df['p1_fs_in_cum'])
        df['p2_w1sp'] = _safe_div(df['p2_fs_won_cum'], df['p2_fs_in_cum'])
        df['p1_w2sp'] = _safe_div(df['p1_ss_won_cum'], df['p1_ss_in_cum'])
        df['p2_w2sp'] = _safe_div(df['p2_ss_won_cum'], df['p2_ss_in_cum'])
    else:
        # create empty columns so downstream code works
        df['p1_fsp'] = 0.0; df['p2_fsp'] = 0.0
        df['p1_w1sp'] = 0.0; df['p2_w1sp'] = 0.0
        df['p1_w2sp'] = 0.0; df['p2_w2sp'] = 0.0
        df['p1_df_cum'] = 0.0; df['p2_df_cum'] = 0.0

    # Optional aces / df cumulatives if present (before current point)
    if {'P1Ace','P2Ace'}.issubset(df.columns):
        df['p1_aces_cum'] = df['P1Ace'].groupby(df['match_id']).cumsum().shift(1).fillna(0)
        df['p2_aces_cum'] = df['P2Ace'].groupby(df['match_id']).cumsum().shift(1).fillna(0)
    else:
        df['p1_aces_cum'] = 0.0; df['p2_aces_cum'] = 0.0

    # ---------- GAMES IN SET (before this point) ----------
    # Determine each game's winner from the LAST point of that game
    last_point_idx = df.groupby(['match_id','SetNo','GameNo']).tail(1).index
    game_winner = pd.Series(index=last_point_idx, data=np.where((p1_won_point+p2_won_point).loc[last_point_idx]==1,
                                                                np.where(p1_won_point.loc[last_point_idx]==1,1,2),
                                                                np.nan), dtype='float')
    # Per-game results table
    games_tbl = df.loc[last_point_idx, ['match_id','SetNo','GameNo']].copy()
    games_tbl['game_winner'] = game_winner.values
    games_tbl['p1_game_win'] = (games_tbl['game_winner'] == 1).astype(int)
    games_tbl['p2_game_win'] = (games_tbl['game_winner'] == 2).astype(int)

    # Cumulative games in the set, then shift so it's "before this game starts"
    games_tbl['p1_games_in_set_cum'] = games_tbl.groupby(['match_id','SetNo'])['p1_game_win'].cumsum()
    games_tbl['p2_games_in_set_cum'] = games_tbl.groupby(['match_id','SetNo'])['p2_game_win'].cumsum()
    games_tbl['p1_games_before'] = games_tbl.groupby(['match_id','SetNo'])['p1_games_in_set_cum'].shift(1).fillna(0).astype(int)
    games_tbl['p2_games_before'] = games_tbl.groupby(['match_id','SetNo'])['p2_games_in_set_cum'].shift(1).fillna(0).astype(int)

    # Map those "before" counts back to every point within each game
    df = df.merge(
        games_tbl[['match_id','SetNo','GameNo','p1_games_before','p2_games_before']],
        on=['match_id','SetNo','GameNo'],
        how='left'
    )

    # ---------- SETS WON BEFORE THIS SET ----------
    # Winner per set = who won more games in that set (from games_tbl at that set's last game)
    set_last_games = games_tbl.groupby(['match_id','SetNo']).tail(1)
    set_last_games = set_last_games.copy()
    set_last_games['set_winner'] = np.where(
        set_last_games['p1_games_in_set_cum'] > set_last_games['p2_games_in_set_cum'], 1, 2
    )
    set_last_games['p1_set_win'] = (set_last_games['set_winner'] == 1).astype(int)
    set_last_games['p2_set_win'] = (set_last_games['set_winner'] == 2).astype(int)
    # Cum sets across match, shift to be "before this set starts"
    set_last_games['p1_sets_before'] = set_last_games.groupby('match_id')['p1_set_win'].cumsum().shift(1).fillna(0).astype(int)
    set_last_games['p2_sets_before'] = set_last_games.groupby('match_id')['p2_set_win'].cumsum().shift(1).fillna(0).astype(int)

    # Join those back to all points within the set
    df = df.merge(
        set_last_games[['match_id','SetNo','p1_sets_before','p2_sets_before']],
        on=['match_id','SetNo'],
        how='left'
    ).fillna({'p1_sets_before':0,'p2_sets_before':0})

    # ---------- BASIC FLAGS ----------
    # is_tiebreak if entering a 6-6 situation (before current game)
    df['is_tiebreak_game'] = ((df['p1_games_before'] == 6) & (df['p2_games_before'] == 6)).astype(int)
    # Server perspective flag (for P1; will flip for P2 later)
    df['server_is_p1'] = (df['PointServer'] == 1).astype(int)

    # ---------- BUILD TWO PERSPECTIVES ----------
    base_cols = ['match_id','SetNo','GameNo','PointNumber','point_idx']
    # P1 perspective
    p1 = df[base_cols].copy()
    p1['perspective'] = 'P1'
    p1['server_is_persp']     = df['server_is_p1']

    p1['pts_in_game_for']     = df['p1_pts_in_game']
    p1['pts_in_game_against'] = df['p2_pts_in_game']
    p1['games_in_set_for']    = df['p1_games_before']
    p1['games_in_set_against']= df['p2_games_before']
    p1['sets_for']            = df['p1_sets_before']
    p1['sets_against']        = df['p2_sets_before']
    p1['is_tiebreak']         = df['is_tiebreak_game']

    # diffs (performance so far)
    p1['ttl_diff'] = df['ttl_p1'] - df['ttl_p2']
    p1['aces_diff'] = df['p1_aces_cum'] - df['p2_aces_cum']
    p1['df_diff'] = df['p1_df_cum'] - df['p2_df_cum']
    p1['fsp_diff'] = df['p1_fsp'] - df['p2_fsp']
    p1['w1sp_diff'] = df['p1_w1sp'] - df['p2_w1sp']
    p1['w2sp_diff'] = df['p1_w2sp'] - df['p2_w2sp']

    # P2 perspective (flip)
    p2 = df[base_cols].copy()
    p2['perspective']         = 'P2'
    p2['server_is_persp']     = 1 - df['server_is_p1']

    p2['pts_in_game_for']     = df['p2_pts_in_game']
    p2['pts_in_game_against'] = df['p1_pts_in_game']
    p2['games_in_set_for']    = df['p2_games_before']
    p2['games_in_set_against']= df['p1_games_before']
    p2['sets_for']            = df['p2_sets_before']
    p2['sets_against']        = df['p1_sets_before']
    p2['is_tiebreak']         = df['is_tiebreak_game']

    p2['ttl_diff'] = -(df['ttl_p1'] - df['ttl_p2'])
    p2['aces_diff'] = -(df['p1_aces_cum'] - df['p2_aces_cum'])
    p2['df_diff']   = -(df['p1_df_cum'] - df['p2_df_cum'])
    p2['fsp_diff']  = -(df['p1_fsp'] - df['p2_fsp'])
    p2['w1sp_diff'] = -(df['p1_w1sp'] - df['p2_w1sp'])
    p2['w2sp_diff'] = -(df['p1_w2sp'] - df['p2_w2sp'])

    panel = pd.concat([p1, p2], ignore_index=True)

    # ---------- PRESSURE FLAGS (computed per perspective) ----------
    panel['is_game_point_for']     = _is_game_point_for(panel['pts_in_game_for'], panel['pts_in_game_against'], panel['is_tiebreak'].astype(bool)).astype(int)
    panel['is_game_point_against'] = _is_game_point_against(panel['pts_in_game_for'], panel['pts_in_game_against'], panel['is_tiebreak'].astype(bool)).astype(int)
    # Break point: receiver can win the game on this point
    panel['is_break_point']        = ((1 - panel['server_is_persp']).astype(bool) & panel['is_game_point_for'].astype(bool)).astype(int)

    # Best-of / sets-needed (constant here; adjust if you have women/Bo3)
    panel['best_of'] = best_of_default
    panel['sets_needed_to_win'] = (panel['best_of'] // 2) + 1

    # Order columns nicely
    ordered = [
        'match_id','SetNo','GameNo','PointNumber','point_idx','perspective',
        'server_is_persp',
        'pts_in_game_for','pts_in_game_against',
        'games_in_set_for','games_in_set_against',
        'sets_for','sets_against',
        'best_of','sets_needed_to_win',
        'is_tiebreak','is_game_point_for','is_game_point_against','is_break_point',
        'ttl_diff','aces_diff','df_diff','fsp_diff','w1sp_diff','w2sp_diff'
    ]
    # Keep only those that actually exist (serve-return diffs may be zeros if not available)
    ordered = [c for c in ordered if c in panel.columns]
    panel = panel[ordered].copy()

    return panel


In [74]:
path ='2012-ausopen-points.csv'
ex = pd.read_csv(f"data/raw data/{path}")
games = ex.sort_values(['match_id','SetNo','GameNo','PointNumber']).copy()
#print(games['match_id'].unique())
for i in (games['match_id'].unique().tolist()):
    print(games[games['match_id'] == i][[
                                        'match_id',
                                        'SetNo',
                                        'GameNo',
                                        'PointNumber',
                                        'PointServer',
                                        'P1Score',
                                        'P2Score',
                                        'P1Momentum',
                                        'P2Momentum',
                                        'P1PointsWon',
                                        'P2PointsWon',
                                        'P1Ace',
                                        'P2Ace',
                                        'P1Winner',
                                        'P2Winner',
                                        ]])
    break

              match_id  SetNo  GameNo  PointNumber  PointServer P1Score  \
0    2012-ausopen-1101      1       1            1            1      15   
1    2012-ausopen-1101      1       1            2            1      30   
2    2012-ausopen-1101      1       1            3            1      40   
3    2012-ausopen-1101      1       1            4            1      40   
4    2012-ausopen-1101      1       1            5            1      40   
..                 ...    ...     ...          ...          ...     ...   
118  2012-ausopen-1101      3       6          119            2       0   
119  2012-ausopen-1101      3       6          120            2      15   
120  2012-ausopen-1101      3       6          121            2      30   
121  2012-ausopen-1101      3       6          122            2      40   
122  2012-ausopen-1101      3       6          123            2       0   

    P2Score  P1Momentum  P2Momentum  P1PointsWon  P2PointsWon  P1Ace  P2Ace  \
0         0         

In [75]:
build_match_state_panel(ex)

/tmp/ipykernel_38599/1697392913.py:36: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df['game_key'] = df.groupby('match_id').apply(


KeyError: 'Column not found: None'